# CICIDS2017 - Détection avec Random Forest

In [20]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import joblib


## 🔹 Chargement des données simulées

In [21]:
# Colonnes utilisées dans le modèle
features = [
    'flow duration',
    'total fwd packets',
    'total backward packets',
    'destination port'
]

expected_cols = features + ['label']

# Initialisation du modèle avec entraînement incrémental (warm_start)
rf = RandomForestClassifier(n_estimators=0, warm_start=True, random_state=42)

# Paramètres de lecture
chunk_size = 10000
n_chunks = 0

In [22]:
## Lecture des données et entrainement progressif

In [23]:
total_rows = 0
for chunk in pd.read_csv('../data/cicids2017.csv', chunksize=chunk_size):
    total_rows += len(chunk)
    print(f"✅ Chunk {n_chunks} traité — total lignes : {total_rows} — arbres : {rf.n_estimators}")

    chunk.columns = chunk.columns.str.strip().str.lower()  # nettoie les noms

    # Nettoyage
    chunk = chunk.dropna().drop_duplicates()
    if 'label' not in chunk.columns:
        print("⛔ Chunk ignoré : colonne 'label' absente")
        continue

    chunk = chunk[chunk['label'].isin(['BENIGN', 'DoS'])]

    # Vérification des colonnes
    missing = [col for col in [f.lower() for f in features + ['label']] if col not in chunk.columns]
    if missing or chunk.empty:
        print(f"⛔ Chunk ignoré : colonnes manquantes {missing}")
        continue

    # Encodage
    chunk['label'] = LabelEncoder().fit_transform(chunk['label'])

    X = chunk[[f.lower() for f in features]]
    y = chunk['label']
    X_scaled = StandardScaler().fit_transform(X)

    rf.n_estimators += 10
    rf.fit(X_scaled, y)

    n_chunks += 1
    print(f"✅ Chunk {n_chunks} traité")


✅ Chunk 0 traité — total lignes : 10000 — arbres : 0
✅ Chunk 1 traité
✅ Chunk 1 traité — total lignes : 20000 — arbres : 10
✅ Chunk 2 traité
✅ Chunk 2 traité — total lignes : 30000 — arbres : 20
✅ Chunk 3 traité
✅ Chunk 3 traité — total lignes : 40000 — arbres : 30
✅ Chunk 4 traité
✅ Chunk 4 traité — total lignes : 50000 — arbres : 40
✅ Chunk 5 traité
✅ Chunk 5 traité — total lignes : 60000 — arbres : 50
✅ Chunk 6 traité
✅ Chunk 6 traité — total lignes : 70000 — arbres : 60
✅ Chunk 7 traité
✅ Chunk 7 traité — total lignes : 80000 — arbres : 70
✅ Chunk 8 traité
✅ Chunk 8 traité — total lignes : 90000 — arbres : 80
✅ Chunk 9 traité
✅ Chunk 9 traité — total lignes : 100000 — arbres : 90
✅ Chunk 10 traité
✅ Chunk 10 traité — total lignes : 110000 — arbres : 100
✅ Chunk 11 traité
✅ Chunk 11 traité — total lignes : 120000 — arbres : 110
✅ Chunk 12 traité
✅ Chunk 12 traité — total lignes : 130000 — arbres : 120
✅ Chunk 13 traité
✅ Chunk 13 traité — total lignes : 140000 — arbres : 130
✅ Chunk

In [24]:
## Sauvegarde du modèle entraîné

In [25]:
joblib.dump(rf, '../model/rf_model.pkl')
print(f"🎯 Entraînement terminé sur {n_chunks} chunks valides — {total_rows} lignes — {rf.n_estimators} arbres")


🎯 Entraînement terminé sur 23 chunks valides — 225745 lignes — 230 arbres
zebi
